In [1]:
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver, NumPyEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

from qiskit.providers.aer import noise
# IBMQ.save_account('991b2673aee106418137b8c38f696ec1d7f503d32b60a09b99c055317110e424b71db909587cc262c804eaac8cb2e3af4004b2c0632ff4965d3fe66358d09f72')
provider = IBMQ.load_account()

import numpy as np
import matplotlib.pyplot as plt
from functools import partial

# Qiskit Summer School Final Project: VQE

#### For this optional final challenge, you will be designing your own implementation of a variational quantum eigensolver (VQE) algorithm that simulates the ground state energy of the Lithium Hydride (LiH) molecule. Through out this challenge, you will be able to make choices on how you want to compose your simulation and what is the final deliverable that you want to showcase to your classmates and friends.

# Defining your molecule:
In this challenge we will focus on LiH using the sto3g basis with the PySCF driver, which can be described in Qiskit as follows, where 'inter_dist' is the interatomic distance.

In [2]:
#driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0' + str(inter_dist), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')

We also setup the molecular orbitals to be considered and can reduce the problem size when we map to the qubit Hamiltonian so the amount of time required for the simulations are reasonable for a laptop computer.

In [3]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order

#### Once you have computed the qubit operations for LiH, you can use the following function to classical solve for the exact solution. This is used just to compare how well your VQE approximation is performing.

In [4]:
#Classically solve for the lowest eigenvalue
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref

Here we ask you to use the `statevector_simulator` as the simulation backend for your VQE algorithm.

In [5]:
backend = BasicAer.get_backend('statevector_simulator')

### Now you can start choosing the components that make up your VQE algorithm!

#### 1. Optimizers
The most commonly used optimizers are `COBYLA`, `L_BFGS_B`, `SLSQP` and `SPSA`. 

#### 2. Qubit mapping
There are several different mappings for your qubit Hamiltonian, `parity`, `bravyi_kitaev`, `jordan_wigner`, which in some cases can allow you to further reduce the problem size.

#### 3. Initial state
There are different initial state that you can choose to start your simulation. Typically people choose from the zero state 
`init_state = Zero(qubitOp.num_qubits)` 
and the UCCSD initial state
`HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type, qubit_reduction)`

#### 4. Parameterized circuit
There are different choices you can make on the form of variational forms of your parameterized circuit.

`UCCSD_var_form = UCCSD(num_qubits, depth=depth, num_orbitals=num_spin_orbitals, num_particles=num_particles)`
    
`RY_var_form = RY(num_qubits, depth=depth)`
    
`RYRZ_var_form = RYRZ(num_qubits, depth=depth)`
    
`swaprz_var_form = SwapRZ(num_qubits, depth=depth)`

#### 5. Simulation backend
There are different simulation backends that you can use to perform your simulation

`backend = BasicAer.get_backend('statevector_simulator')`

`backend=Aer.get_backend('qasm_simulator')`

### Compare the convergence of different choices for building your VQE algorithm

Among the above choices, which combination do you think would out perform others and give you the lowest estimation of LiH ground state energy with the quickest convergence? Compare the results of different combinations against each other and against the classically computed exact solution at a fixed interatomic distance, for example `inter_dist=1.6`. 

To access the intermediate data during the optimization, you would need to utilize the `callback` option in the VQE function:

`def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)
            params.append(parameters)
            deviation.append(std)`
            
`algo = VQE(qubitOp, var_form, optimizer, callback=store_intermediate_result)`

`algo_result = algo.run(quantum_instance)`

An example of comparing the performance of different optimizers while using the RY variational ansatz could like the following:
![RY_error.png](attachment:RY_error.png)
![RY_convergence.png](attachment:RY_convergence.png)

### Compute the ground state energy of LiH at various different interatomic distances
By changing the parameter `inter_dist`, you can use your VQE algorithm to calculate the ground state energy of LiH at various interatomic distances, and potentially produce a plot as you are seeing here. Note that the VQE results are very close to the exact results, and so the exact energy curve is hidden by the VQE curve.
<img src="attachment:VQE_dist.png" width="600">

### How does your VQE algorithm perform in the presence of noise?
Trying importing the noise model and qubit coupling map of a real IBM quantum device into your simulation. You can use the imported noise model in your simulation by passing it into your quantum instance. You can also try enabling error mitigation in order to lower the effect of noise on your simulation results.

In [6]:
#Define our noise model based on the ibmq_essex chip
chip_name = 'ibmq_essex'
device = provider.get_backend(chip_name)
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(device.properties())
basis_gates = noise_model.basis_gates

/home/nemo/.local/lib/python3.6/site-packages/qiskit/providers/aer/noise/device/basic_device_model.py:115: DeprecationWarning: This function is been deprecated and moved to a method of the`NoiseModel` class. For equivalent functionality use `NoiseModel.from_backend(properties, **kwargs).
  DeprecationWarning)


An example of comparing the energy convergence of using SPSA and COBYLA with the ibmq_essex noise model could look like the following
![noise.png](attachment:noise.png)

### Now given the choices you have made above, try writing your own VQE algorithm in Qiskit. You can find an example of using Qiskit to simuate molecules with VQE [here](https://qiskit.org/textbook/ch-applications/vqe-molecules.html).

In [48]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# Classically solve for the lowest eigenvalue
# This is used just to compare how well you VQE approximation is performing
def exact_solver(qubitOp):
    #ee = ExactEigensolver(qubitOp)
    ee = NumPyEigensolver(qubitOp)
    result = ee.run()
    #ref = result['energy']
    ref = np.real(result['eigenvalues'][0])
    print('Reference value: {}'.format(ref))
    return ref

# Define your function for computing the qubit operations of LiH
def compute_LiH_qubitOp(map_type, inter_dist, basis='sto3g'):
    
    # Specify details of our molecule
    driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 ' + str(inter_dist), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis=basis)

    # Compute relevant 1 and 2 body integrals.
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))

    # Please be aware that the idx here with respective to original idx
    freeze_list = [0]
    remove_list = [-3, -2] # negative number denotes the reverse order
    
    # Prepare full idx of freeze_list and remove_list
    # Convert all negative idx to positive
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    
    # Update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]

    # Prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
    # and if PARITY mapping is selected, reduction qubits
    energy_shift = 0.0
    qubit_reduction = True if map_type == 'parity' else False

    ferOp = FermionicOperator(h1=h1, h2=h2)
    if len(freeze_list) > 0:
        ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
        num_spin_orbitals -= len(freeze_list)
        num_particles -= len(freeze_list)
    if len(remove_list) > 0:
        ferOp = ferOp.fermion_mode_elimination(remove_list)
        num_spin_orbitals -= len(remove_list)

    qubitOp = ferOp.mapping(map_type=map_type)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
    qubitOp.chop(10**-10)

    return qubitOp, num_spin_orbitals, num_particles, qubit_reduction

map_type = 'parity'
inter_dist = 1.6

qubitOp, num_spin_orbitals, num_particles, qubit_reduction = compute_LiH_qubitOp(map_type, inter_dist)
# Classically solve for the exact solution and use that as your reference value
ref = exact_solver(qubitOp)

# Specify your initial state
init_state =  HartreeFock(num_spin_orbitals, num_particles, map_type, qubit_reduction)#Zero(qubitOp.num_qubits)#

# Select a state preparation ansatz
# Equivalently, choose a parameterization for our trial wave function.

from qiskit.circuit.library import EfficientSU2

# var_form = EfficientSU2(num_qubits=qubitOp.num_qubits, 
#                        entanglement='sca', 
#                        reps=2, 
#                        skip_final_rotation_layer=True, 
#                        initial_state=init_state
#                        )
var_form = RYRZ(num_qubits=qubitOp.num_qubits, 
                     depth=3, 
                     entangler_map=[[0,1], [1,2], [1,3]], 
                     entanglement='linear',
                     initial_state=init_state,
                     entanglement_gate='cz', 
                     skip_unentangled_qubits=False)

# var_form =UCCSD(
#         num_orbitals=num_spin_orbitals,
#         num_particles=num_particles,
#         initial_state=init_state,
#         qubit_mapping='parity')

    
# Choose where to run/simulate our circuit
# backend = Aer.get_backend('qasm_simulator')

# from qiskit.ignis.mitigation import CompleteMeasFitter
# quantum_instance = QuantumInstance(backend=backend, 
#                                    shots=1000,
#                                    noise_model=noise_model, 
#                                    coupling_map=coupling_map,
#                                    measurement_error_mitigation_cls=CompleteMeasFitter,
#                                    basis_gates=basis_gates,
#                                    optimization_level=3
#                                   )
backend = Aer.get_backend('qasm_simulator')
#     backend = QasmSimulator()
#     backend_options = {"method": "statevector_gpu"}

quantum_instance = QuantumInstance(backend=backend,
                               backend_options={
#                                                "optimize_noise_threshold":4,
                                               "zero_threshold":1e-3,
                                                "max_parallel_experiments":0,
#                                                "optimize_ideal_threshold":4
                                               },
                               shots=1024,
                               noise_model=noise_model, 
                               coupling_map=coupling_map,
                               measurement_error_mitigation_cls=CompleteMeasFitter,
                               basis_gates=basis_gates,
                               cals_matrix_refresh_period=30,
                               optimization_level=3,
                               )


# Choose the classical optimizer
# optimizer = COBYLA(maxiter=600, tol=0.0001)
#optimizer = SLSQP(maxiter=600)

# optimizer = COBYLA(maxiter=300, tol = 1e-04)

optimizer = L_BFGS_B(maxiter=50, epsilon=1e-3)
# optimizer = SPSA(max_trials= 100, last_avg = 5,c0=0.6283185307179586, c1=0.1, c2=0.101, c3=0.101, c4=0,)

# Run your VQE instance
vqe = VQE(qubitOp, var_form, optimizer, quantum_instance=quantum_instance)

import time
start = time.time()

# Now compare the results of different compositions of your VQE algorithm!
vqe_result = np.real(vqe.run(quantum_instance)['eigenvalue'])
print('Energy found   :',vqe_result)

end = time.time()
print(end - start,'seconds')


HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12
Reference value: -1.0770597457346944
Energy found   : -0.45592034476085
294.8932943344116 seconds


In [ ]:
UCCSD_var_form = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, qubit_mapping='parity')
print(UCCSD_var_form.get_entangler_map(map_type='linear',num_qubits=qubitOp.num_qubits))
swaprz_var_form = SwapRZ(qubitOp.num_qubits)
print(swaprz_var_form.get_entangler_map(map_type='linear',num_qubits=qubitOp.num_qubits))

UCCSD_var_form.validate_entangler_map([[0, 1], [1, 2], [1, 3]], qubitOp.num_qubits)
UCCSD_var_form.get_entangler_map(map_type='linear',num_qubits=qubitOp.num_qubits)
#print(var_form.setting)

[[0, 1], [1, 2], [2, 3]]
[[0, 1], [1, 2], [2, 3]]


[[0, 1], [1, 2], [2, 3]]

In [47]:
ref

-1.07705974573469

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright